# Introduction

Last run: 2020-04-09

The Covid-19 pandemic has turned into a global emergency and is now affecting our lives in the past few months. In the relatively privileged community that is the Kaggle community (people with a certain level of education and internet access), the collective effort to maintain datasets, create insights, and contribute to the research is admirable and, being the only thing I can do to help out, this notebook is my attempt of contributing to that effort.

As an Italian living in the Netherlands, I have observed my Country underestimating the risk, then taking it very seriously, and now facing a daily tragedy with a strict lockdown and critical situations all over the Country. At the same time, I could see very similar behavior in the Country I live in and in others. As if the news from a near Country were not to be taken as an example to avoid the worst.

This notebook is an attempt to show with data how similar and how different the spread of the pandemic in different countries is. To do so, we have will rely on official data. We have to keep in mind, however, that the data is not gathered uniformly in every Country and that every Country has a different nuance in counting cases or deceased patients. For example, if Italy was counting as Coronavirus death only the patients without pre-existing illnesses, the death count would be [only 1% of the current value](https://www.bloomberg.com/news/articles/2020-03-18/99-of-those-who-died-from-virus-had-other-illness-italy-says). Or if a Country does not perform tests on purpose as they are interested in the spreading and not in the actual count, their number will be unrealistically low when compared with other countries.

Moreover, we have no way of taking into account other factors, like deaths due to the hospital being too busy with Covid-19 patients, or patients not finding their prescribed drugs for other diseases because someone on the internet said that those drugs were curing Covid-19, or the future deaths due to the economic crisis that this outbreak is causing.

All this to say that every analysis is as good as the data is based on and we should always question the nature of that data, especially in a delicate situation like this one. This should not stop me from thanking a few users for their effort in maintaining clean datasets and constantly trying to fix mistakes in them. In particular [@sudalairajkumar](https://www.kaggle.com/sudalairajkumar) and [@paultimothymooney](https://www.kaggle.com/paultimothymooney).

This notebook is organized as follows (and it will be a constant work in progress until the pandemic is over)
* How is the pandemic growing? Trends by Country and a picture of how we got here.
* The Italian case. Region by Region analysis of containment measures, spread, and death rates
* Similar Countries. Which Countries are showing the same trends and which ones are very different from the others?
* Did the rest of the world learn something? Given the Italian situation, what did the other Countries do?

By combining a few sources of data, we will work on datasets that look like this

In [ ]:
import numpy as np
import pandas as pd 
from scipy.optimize import curve_fit

from datetime import timedelta

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
def min_date(data, num, text, by=None):
    
    if by is None:
        by = 'Country_Region'
    
    min_dates = data[data['n_cases'] > num].groupby(by, as_index=False).date.min()
    
    return min_dates.rename(columns={'date': text})


def increase(data):
    increase = data[['Country_Region', 'date', 'n_cases']].sort_values(['Country_Region', 'date'], ascending=True).copy()
    increase['increase'] = increase.n_cases.diff().fillna(0)
    increase.loc[increase.increase < 0, 'increase'] = 0
    increase['perc_increase'] = (increase.increase / (increase.n_cases - increase.increase) * 100).fillna(0)
    
    return increase[['Country_Region', 'date', 'increase', 'perc_increase']]


def load_series(path):
    ignore = ['Province/State', 'Lat', 'Long']
    data = pd.read_csv(path)
    
    data = data[[col for col in data if col not in ignore]].groupby('Country/Region', as_index=False).sum()
    data.rename(columns={'Country/Region': 'Country_Region'}, inplace=True)
    
    data = data.melt(id_vars='Country_Region')
    data['date'] = pd.to_datetime(data['variable'])
    del data['variable']
    data.rename(columns={'value': 'n_cases'}, inplace=True)
    
    first_case = min_date(data, 0, 'first_date')
    data = pd.merge(data, first_case, on='Country_Region', how='left')
    data['from_first'] = (data['date'] - data['first_date']).dt.days
    
    case_10 = min_date(data, 9, '10th_date')
    data = pd.merge(data, case_10, on='Country_Region', how='left')
    data['from_10th'] = (data['date'] - data['10th_date']).dt.days
    
    case_50 = min_date(data, 49, '50th_date')
    data = pd.merge(data, case_50, on='Country_Region', how='left')
    data['from_50th'] = (data['date'] - data['50th_date']).dt.days
    
    case_100 = min_date(data, 99, '100th_date')
    data = pd.merge(data, case_100, on='Country_Region', how='left')
    data['from_100th'] = (data['date'] - data['100th_date']).dt.days
    
    case_500 = min_date(data, 499, '500th_date')
    data = pd.merge(data, case_500, on='Country_Region', how='left')
    data['from_500th'] = (data['date'] - data['500th_date']).dt.days
    
    data['Country_Region'] = data['Country_Region'].map({'US': 'United States', 
                                                         'Korea, South': 'South Korea'}).fillna(data['Country_Region'])
    
    continents = pd.read_csv('/kaggle/input/country-to-continent/countryContinent.csv', encoding = 'ISO-8859-1')
    continents['country'] = continents['country'].map({'United States of America': 'United States', 
                                                       'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
                                                       'Korea (Republic of)': 'South Korea', 
                                                       "Korea (Democratic People's Republic of)": 'North Korea'}).fillna(continents['country'])
    
    data = pd.merge(data, continents[['country', 'continent', 'sub_region']], left_on='Country_Region', right_on='country', how='left')
    del data['country']
    
    country_info = pd.read_csv('/kaggle/input/population-by-country-2020/population_by_country_2020.csv')
    data = pd.merge(data, country_info[['Country (or dependency)', 'Population (2020)', 'Density (P/Km²)', 'Med. Age']], 
                    left_on='Country_Region', right_on='Country (or dependency)', how='left')
    data.rename(columns={'Population (2020)': 'population', 'Density (P/Km²)': 'pop_density', 'Med. Age': 'median_age'}, inplace=True)
    data.loc[data.median_age == 'N.A.', 'median_age'] = np.nan
    data['median_age'] = pd.to_numeric(data.median_age)
    del data['Country (or dependency)']
    
    new_cases = increase(data)
    data = pd.merge(data, new_cases, on=['Country_Region', 'date'], how='left')
    
    return data


def start_from(cases, deaths, col_start, on_cases=False, n_top=10, true_count='x', make_rate=False, ch_date=True):
    if on_cases:
        tmp = pd.merge(cases[['Country_Region', 'date', 'continent', 'population', 'n_cases']+[col_start]], 
                      deaths[['Country_Region', 'date', 'n_cases']], on=['Country_Region', 'date'])
    else:
        tmp = pd.merge(cases[['Country_Region', 'date', 'continent', 'population', 'n_cases']], 
                      deaths[['Country_Region', 'date', 'n_cases']+[col_start]], on=['Country_Region', 'date'])
        
    top_countries = tmp.groupby('Country_Region').n_cases_x.max().sort_values(ascending=True).tail(n_top).index.tolist()
    tmp = tmp[tmp[col_start] > 0]
    tmp = tmp[tmp.Country_Region.isin(top_countries)]
    if ch_date:
        tmp['date'] = tmp[col_start]
    if make_rate:
        tmp['n_cases_y'] = (tmp['n_cases_y'] / tmp['n_cases_x'] * 100).fillna(0)
        true_count = 'y'
    
    tmp['n_cases'] = tmp[f'n_cases_{true_count}']
    
    return tmp


def country_cont(data, country):
    df = data[data.Country == country][['Date Start',
                                    'Description of measure implemented',
                                    'Exceptions', 
                                    'Keywords', 
                                    'Target region']].copy()
    df['region'] = df['Target region'].fillna('All')
    df['Keywords'] = df['Keywords'].fillna('-')
    df['date'] = pd.to_datetime(df['Date Start'])
    df.drop(['Date Start', 'Target region'], axis=1, inplace=True)
    df = df.sort_values(by='date').reset_index(drop=True)
    
    return df

def process_germany(data_loc, pop_loc):
    germany = pd.read_csv(data_loc)
    germ_pop = pd.read_csv(pop_loc)
    
    germany['date'] = pd.to_datetime(germany['date']).dt.date
    germany['gender'] = germany['gender'].map({'M': 'male', 'F': 'female'})
    germany = germany.groupby(['state', 'age_group', 'gender', 'date'], as_index=False).sum()
    
    germany = pd.merge(germany, germ_pop, on=['state', 'age_group', 'gender'], how='left').sort_values('date')
    
    germany.rename(columns={'cases': 'new_cases', 'deaths': 'new_deaths'}, inplace=True)
    
    germany['total_cases'] = germany.groupby(['state', 'age_group', 'gender', 'population'], as_index=False).new_cases.cumsum()
    germany['total_deaths'] = germany.groupby(['state', 'age_group', 'gender', 'population'], as_index=False).new_deaths.cumsum()
    
    return germany.reset_index(drop=True)

In [ ]:
conf_cases = load_series('/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv')
recovered = load_series('/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv')
deaths = load_series('/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv')

# containment = pd.read_csv('/kaggle/input/covid19-containment-and-mitigation-measures/COVID%2019%20Containment%20measures%202020-03-30.csv')
# tmp = pd.read_csv('/kaggle/input/covid19-containment-and-mitigation-measures/COVID-19%20containment%20measures/COVID 19 Containment measures data.csv')
# containment = pd.concat([containment, tmp], ignore_index=True)
# containment.loc[containment.Country.fillna('-').str.contains('US:'), 'Country'] = 'United States'
containment = pd.read_csv('/kaggle/input/covid19-containment-and-mitigation-measures/COVID 19 Containment measures data.csv')
containment.loc[containment.Country.fillna('-').str.contains('US:'), 'Country'] = 'United States'

germany = process_germany('/kaggle/input/covid19-tracking-germany/covid_de.csv', '/kaggle/input/covid19-tracking-germany/demographics_de.csv')

conf_cases.head()

In [ ]:
conf_cases.date.max()

diff:@1:3, diff:plugins/0.12/mercurial-plugin@9128:9953, diff:tags/trac-0.9.2/wiki-default//tags/trac-0.9.3/wiki-default or diff:trunk/trac@3538//sandbox/vc-refactoring@3539

# How is the pandemic growing?

We will now see how the disease spread across the world since Jan. 22nd 2020. We will focus on the 3 data sources that we have: Confirmed Cases, Recovered Patients, and Deceased Patiens. We will focus both on absolute numbers and on increase ratios.

In [ ]:
def plot_time_count(data, title, groups_time=None, groups_count=None, legend=True, increase=False):    
    fig = plt.figure(figsize=(18, 8), facecolor='#f7f7f7') 
    fig.suptitle(title, fontsize=18)

    spec = fig.add_gridspec(ncols=2, nrows=1, width_ratios=[5, 2])

    ax0 = fig.add_subplot(spec[0])
    ax1 = fig.add_subplot(spec[1])
    
    if increase:
        use_col = 'increase'
    else:
        use_col = 'n_cases'
    
    if groups_time is None:
        groups_time = ['date']
    if groups_count is None:
        groups_count = 'Country_Region'
    
    tmp = data.groupby(groups_time,as_index=False)[use_col].sum().set_index('date')
    if len(groups_time) > 1:
        tmp.groupby([c for c in groups_time if c != 'date'])[use_col].plot(ax=ax0, legend=legend)
    else:   
        tmp[use_col].plot(ax=ax0, legend=legend)
    
    tmp = data[data.date == data.date.max()].groupby(groups_count)[use_col].sum().sort_values(ascending=True).tail(10)
    tmp.plot(ax=ax1, kind='barh')
    ax1.grid(axis='x')
    
    ax1.set_ylabel('')
    ax0.set_xlabel('')
    
    plt.show()
    
    
def plot_resample(data, column, ax, color, label, index=True, resample='3D'):
    if index:
        res = pd.Series(data[column], index=pd.to_datetime(data.index))
    else:
        res = pd.Series(data[column], index=pd.to_datetime(data['date']))
    
    res.resample(resample).mean().plot(ax=ax, color=color, label=label)
    
    return ax


def make_areas(ax, max_date, offset=7):
    ax.axvline(pd.to_datetime('2020-03-1'), color='y', linestyle='--')
    ax.axvline(pd.to_datetime('2020-03-9'), color='r', linestyle='--')
    ax.axvline(pd.to_datetime('2020-03-20'), color='m', linestyle='--')
    ax.axvspan(pd.to_datetime('2020-03-1') + timedelta(days=offset), max_date, facecolor='y', alpha=0.2)
    ax.axvspan(pd.to_datetime('2020-03-9') + timedelta(days=offset), max_date, facecolor='r', alpha=0.2)
    ax.axvspan(pd.to_datetime('2020-03-20') + timedelta(days=offset), max_date, facecolor='m', alpha=0.2)
    
    return ax


def get_avg_beforemeasure(data, col, ax):
    avg_beforeyellow = data[(data.index > pd.to_datetime('2020-03-1') - timedelta(days=7)) & 
                           (data.index < pd.to_datetime('2020-03-1'))][col].mean()
    avg_beforered = data[(data.index > pd.to_datetime('2020-03-9') - timedelta(days=7)) & 
                           (data.index < pd.to_datetime('2020-03-9'))][col].mean()
    avg_beforepurple = data[(data.index > pd.to_datetime('2020-03-20') - timedelta(days=7)) & 
                           (data.index < pd.to_datetime('2020-03-20'))][col].mean()
    
    ax[2].axhline(avg_beforeyellow, color='y', linestyle='--', alpha=0.5)
    ax[2].axhline(avg_beforered, color='r', linestyle='--', alpha=0.5)
    ax[2].axhline(avg_beforepurple, color='m', linestyle='--', alpha=0.5)
    
    return ax


def expon(x, x0, k, i):
    y = x0 * np.exp(k*x) + i
    return y


def plot_fits(data, col, ax, offset=7):
    df = data.reset_index()
    
    #df[col] = df[col] - df[col].min()
    
    dates = {'2020-03-1': ['y', 'No Yellow Zone'],
             '2020-03-9': ['r', 'No Red Zone'], 
             '2020-03-20': ['m', 'No Purple Zone']}
    
    for day in dates.keys():
        tmp_df = df[df.Date < (pd.to_datetime(day) + timedelta(days=offset))]
        n_days = (pd.to_datetime(day) - pd.to_datetime(df.Date.min())).days + 1
        xdata = np.array(tmp_df.index)
        ydata = np.array(tmp_df[col])
        popt, pcov = curve_fit(expon, xdata, ydata)
        x = np.linspace(-1, df.index.max(), 100)
        y = expon(x, *popt)

        ax.plot(xdata, ydata, 'o', color='k')
        ax.plot(x, y, label=dates[day][1], color=dates[day][0])
        ax.axvline(n_days, color=dates[day][0], linestyle='--')
    
    return ax


def plot_country(data, country, col, ax, color, cases=False):
    tmp = data[data.Country_Region==country]
    if cases:
        ax.scatter(tmp[col], tmp.n_cases/tmp.population*1000, s=tmp.population/1000000, color=color)
    else:
        ax.scatter(tmp[col], tmp.n_cases, s=tmp.population/1000000, color=color)
    return ax

## Confirmed cases

In reading this section, please keep in mind that the numbers are heavily influenced by how each Country is conducting tests and reporting the numbers.

The Country with more confirmed cases was China, which is not surprising since it is the first Country with confirmed cases and the one with the largest population. Unfortunately the USA has now passed that number in a very short time after the outbreak on their territory and other European countries, with a substanstially smaller population, has passed that number as well.

In [ ]:
plot_time_count(conf_cases, 'Total Confirmed Cases', ['date'], legend=False)

Breaking down the trend by continent, we notice that Asia is essentially reaching the flat part of the curve, while Europe (now the continent with the most confirmed cases) and the Americas are still rapidly increasing their numbers and passed Asia on March 28th. Very few cases reported in Oceania (expected due to the relatively low population size) and Africa (suspiciously low).

The numbers in Asia are overwhelmingly dominated by China that, after very rigid containment measures, appears to be close to spot the spreading of the disease. On the other hand, the numbers in the Americas are dominated by the USA that is where the outbreak is particularly intense at the moment.

In [ ]:
plot_time_count(conf_cases, 'Total Confirmed Cases by Continent', ['date', 'continent'], ['continent'], legend=True)

We can indeed how the daily variation is dominated by Europe since March. From March 25th to April 3rd the growth in Europe has been sustantially flat and the Americas have reached that level on April 2nd. However, after that day a large number of new cases was reported in Europe. After that, the Americas kept being the continent with most new cases in the past day.

In [ ]:
plot_time_count(conf_cases, 'New Confirmed Cases by Continent', ['date', 'continent'], ['continent'], legend=True, increase=True)

In Europe, the situation is getting worse and worse with Italy and Spain sadly leading the way for confirmed cases. We also see how the sudden increase in the number of cases after April 3rd is mainly coming from France. The reason is that the cases in the nursing homes were reported all at once that day.

In [ ]:
tmp = conf_cases[conf_cases.continent == 'Europe']
top_countries = tmp.groupby('Country_Region').n_cases.max().sort_values(ascending=True).tail(10).index.tolist()
tmp = tmp[tmp.Country_Region.isin(top_countries)]

plot_time_count(tmp, 'Total Confirmed Cases in Europe', ['date', 'Country_Region'], legend=True)

However, it is good to keep in mind that if we consider the number of confirmed cases per thousand inhabitants the difference between the top countries (with at least a 1000000 inhabitants) are much smaller and Switzerland and Spain surpass Italy

In [ ]:
tmp = conf_cases.copy()
tmp['n_cases'] = tmp['n_cases'] / tmp['population'] * 1000
tmp = tmp[(tmp.continent == 'Europe') & (tmp.population > 1000000)]
top_countries = tmp.groupby('Country_Region').n_cases.max().sort_values(ascending=True).tail(10).index.tolist()
tmp = tmp[tmp.Country_Region.isin(top_countries)]

plot_time_count(tmp, 'Total Confirmed Cases per 1000 inhabitants in Europe', ['date', 'Country_Region'], legend=True)

Looking at Africa, we see the rapid increase in the number of cases in Egypt, South Africa, and Algeria. Impressive the sudden increase in the number of cases in South Africa

In [ ]:
tmp = conf_cases[conf_cases.continent == 'Africa']
top_countries = tmp.groupby('Country_Region').n_cases.max().sort_values(ascending=True).tail(10).index.tolist()
tmp = tmp[tmp.Country_Region.isin(top_countries)]

plot_time_count(tmp, 'Total Confirmed Cases in Africa', ['date', 'Country_Region'], legend=True)

Regarding the Americas, the situation is widely dominated by the USA, even when we factor the population size out

In [ ]:
tmp = conf_cases.copy()
tmp['n_cases'] = tmp['n_cases'] / tmp['population'] * 1000
tmp = tmp[(tmp.continent == 'Americas') & (tmp.population > 1000000)]
top_countries = tmp.groupby('Country_Region').n_cases.max().sort_values(ascending=True).tail(10).index.tolist()
tmp = tmp[tmp.Country_Region.isin(top_countries)]

plot_time_count(tmp, 'Total Confirmed Cases per 1000 inhabitants in the Americas', ['date', 'Country_Region'], legend=True)

In Asia, on the other hand, we notice how stable is the number of positive cases in South Korea, which has shown great efficiency in containing the spread of the disease. China does not display here as the population is very large and thus their numbers by 1000 inhabitants are low.

In [ ]:
tmp = conf_cases.copy()
tmp['n_cases'] = tmp['n_cases'] / tmp['population'] * 1000
tmp = tmp[(tmp.continent == 'Asia') & (tmp.population > 1000000)]
top_countries = tmp.groupby('Country_Region').n_cases.max().sort_values(ascending=True).tail(10).index.tolist()
tmp = tmp[tmp.Country_Region.isin(top_countries)]

plot_time_count(tmp, 'Total Confirmed Cases per 1000 inhabitants in Asia', ['date', 'Country_Region'], legend=True)

Before moving to the recovery rate, it is worth noticing the similarity of the curves if we consider the days after the 10th deceased patient. This fairly arbitrary limit is to capture the moment when a nation starts *taking the emergency seriously*.

In [ ]:
tmp = start_from(conf_cases, deaths, 'from_10th', on_cases=False, n_top=12, true_count='x')

fig, ax = plt.subplots(1,1, figsize=(18, 8), facecolor='#f7f7f7') 
fig.suptitle('Confirmed cases since the 10th deceased patient', fontsize=18)

tmp = tmp.groupby(['date', 'Country_Region'],as_index=False).n_cases.sum().set_index('date')
tmp.groupby([c for c in ['date', 'Country_Region'] if c != 'date']).n_cases.plot(ax=ax, legend=True)

ax.annotate('USA',
            xy=(tmp[tmp.Country_Region=='United States'].reset_index().date.max(), tmp[tmp.Country_Region=='United States'].n_cases.max()), 
            xycoords='data', xytext=(-55, 0), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('Italy',
            xy=(tmp[tmp.Country_Region=='Italy'].reset_index().date.max(), tmp[tmp.Country_Region=='Italy'].n_cases.max()), 
            xycoords='data', xytext=(20, 20), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('Spain',
            xy=(tmp[tmp.Country_Region=='Spain'].reset_index().date.max(), tmp[tmp.Country_Region=='Spain'].n_cases.max()), 
            xycoords='data', xytext=(-25, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
# ax.annotate('S. Korea',
#             xy=(tmp[tmp.Country_Region=='South Korea'].reset_index().date.max(), tmp[tmp.Country_Region=='South Korea'].n_cases.max()), 
#             xycoords='data', xytext=(-25, 25), textcoords='offset points',
#             arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('Germany',
            xy=(tmp[tmp.Country_Region=='Germany'].reset_index().date.max(), tmp[tmp.Country_Region=='Germany'].n_cases.max()), 
            xycoords='data', xytext=(-25, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('Netherlands',
            xy=(tmp[tmp.Country_Region=='Netherlands'].reset_index().date.max(), tmp[tmp.Country_Region=='Netherlands'].n_cases.max()), 
            xycoords='data', xytext=(25, -25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('UK',
            xy=(tmp[tmp.Country_Region=='United Kingdom'].reset_index().date.max(), tmp[tmp.Country_Region=='United Kingdom'].n_cases.max()), 
            xycoords='data', xytext=(25, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('China',
            xy=(40, tmp[tmp.Country_Region=='China'].n_cases.max()- 1000), 
            xycoords='data', xytext=(-25, -25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('France',
            xy=(tmp[tmp.Country_Region=='France'].reset_index().date.max(), tmp[tmp.Country_Region=='France'].n_cases.max()), 
            xycoords='data', xytext=(25, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))

ax.set_xlabel('Days after 10th deceased patient', fontsize=14)
ax.set_xlim(1,45)

plt.show()

We see how Spain is following a steeper curve than China and how Italy is doing the same with an offset of a few days (but without slowing down like China). Impressive work of containment done by South Korea, which was one of the most affected countries but managed to drop off the above plot. The growth in the USA, on the other hand, looks very scary and is currently unmatched by any other country. The growth in Germany is also very steep but we should consider that the 10th death came very late in the outbreak, so they were already on another part of the curve. The Netherlands has been slowing down in the past few days and got away from the Italian curve, thus following the Iranian one.

## Recovered Patients

This is a tricky section. We know that most of the people with Coronavirus experience mild symptoms and that it is very unlikely that people with mild symptoms get tested and thus end up in this statistic when they fully recover. Looking at the absolute number of recovered patients, China is leading the way, which makes sense for a Country that first experienced the virus and already reached the flat part of the curve.

In [ ]:
plot_time_count(recovered, 'Total Recovered Patients', ['date'], legend=False)

However, these numbers make sense only when compared with the number of confirmed cases. Let's thus focus on the countries with most cases and see how the recovery rate evolved over time. We will consider only the time passed since the 100th confirmed case to avoid out of scale fluctuations (this is why some plots start from a different date)

In [ ]:
tmp = start_from(conf_cases, recovered, 'from_100th', 10, make_rate=True, ch_date=False)

plot_time_count(tmp, 'Recovery rate after 100th confirmed case', ['date', 'Country_Region'], legend=True)

It appears there is some reporting issue, for example France has a nearly 0 recovery rate until March 20th when it goes to about 15%. We see that China has been recovering more and more patients, passing the 20% recovery rate about a month after the 100th patient, which gives good hopes to the other countries. If we indeed align the plots to have them starting at the same day (thus the starting date is whenever these 10 countries had the 100th confirmed case), we get

In [ ]:
tmp = start_from(conf_cases, recovered, 'from_100th', 10, make_rate=True)

fig, ax = plt.subplots(1,1, figsize=(18, 8), facecolor='#f7f7f7') 
fig.suptitle('Recovery rate from 100th patient', fontsize=18)

tmp = tmp.groupby(['date', 'Country_Region'],as_index=False).n_cases.sum().set_index('date')
tmp.groupby([c for c in ['date', 'Country_Region'] if c != 'date']).n_cases.plot(ax=ax, legend=True)

ax.set_xlabel('Days after 100th confirmed case', fontsize=14)
diff:@1:3, diff:plugins/0.12/mercurial-plugin@9128:9953, diff:tags/trac-0.9.2/wiki-default//tags/trac-0.9.3/wiki-default or diff:trunk/trac@3538//sandbox/vc-refactoring@3539


plt.show()



We see how Iran is having a recovery rate much ahead of the curve of China, while Italy looks like is staying behind. We can probably expect to see the recovery rates of the USA and the UK to go up in the next few days but it is evident that this type of observation is reported very inconsistently.

Inconsistencies aside, we can see how many days pass between the confirmation of the case and the recovery. Since we don't have individual-level data, we can compare the number of days between the first case with the first recovery, the tenth case with the tenth recovery, and so on.

This is not necessarily the best way 

In [ ]:
tmp = pd.merge(conf_cases[['Country_Region', 'date', 'continent', 'n_cases', 'first_date', '10th_date', '50th_date', '100th_date']], 
               recovered[['Country_Region', 'date', 'continent', 'n_cases', 'first_date', '10th_date', '50th_date', '100th_date']], 
               on=['Country_Region', 'date', 'continent'])

top_countries = tmp.groupby('Country_Region').n_cases_x.max().sort_values(ascending=True).tail(12).index.tolist()

tmp = tmp[(tmp.Country_Region.isin(top_countries)) & (tmp.Country_Region != 'China')]

tmp['1st recovered/confirmed'] = (tmp['first_date_y'] - tmp['first_date_x']).dt.days
tmp['10th recovered/confirmed'] = (tmp['10th_date_y'] - tmp['10th_date_x']).dt.days
tmp['50th recovered/confirmed'] = (tmp['50th_date_y'] - tmp['50th_date_x']).dt.days
tmp['100th recovered/confirmed'] = (tmp['100th_date_y'] - tmp['100th_date_x']).dt.days

fig, ax = plt.subplots(1,1, figsize=(10, 18), facecolor='#f7f7f7')
fig.suptitle('Days between confirmed case and recovery', fontsize=18)
fig.subplots_adjust(top=0.95)

tmp.groupby('Country_Region')[['1st recovered/confirmed', '10th recovered/confirmed', 
                               '50th recovered/confirmed', '100th recovered/confirmed']].min().plot(kind='barh', ax=ax)

ax.axvline(tmp.groupby('Country_Region')[['1st recovered/confirmed', 
                                          '10th recovered/confirmed', 
                                          '50th recovered/confirmed', 
                                          '100th recovered/confirmed']].min().mean().mean(), linestyle='--', color='k')

ax.set_xlabel('Days from case to recovey', fontsize=14)

plt.show()

We see that on average the recovery occurs in 15 days (mind that this is also influenced by the definition of recovery, as well as the quickness of the laboratories to return the tests' results). We excluded China as when the dataset starts they are well beyond the 100th case already. 

Interesting to see how the USA and the Netherlands have a slower recovery and the extremely quick one of Iran or Italy. Most likely the access to tests and the recovery definition are playing a big role here.

## Deceased Patients

We now get to the more painful section of this first analysis. All the limitations expressed in the previous section still apply.

In [ ]:
plot_time_count(deaths, 'Total Deceased Patients', ['date'], legend=False)

We see that indeed there are 2 main phases, the first one when the virus was claiming victims in Asia, again mainly in China, and a second one when the European situation got critical. We see the beginning of the curve for the Americas and the fear is that they follow the curve of Europe.

In [ ]:
plot_time_count(deaths, 'Total Deceased Patients by Continent', ['date', 'continent'], ['continent'], legend=True)

On the two extremes of the most affected countries we find the very high death rate of Italy (around 12%) and the extremely low one of Germany (well below 2%). Again, this is very dependent on the criteria used to define a death *for* Coronavirus or *with* Coronavirus.

We again consider only the cases after the 500th confirmed one to avoid unrealistic fluctuations (and we still see the very high rate of Iran in the first days)

In [ ]:
tmp = start_from(conf_cases, deaths, 'from_500th', n_top=10, on_cases=True, make_rate=True, ch_date=False)

plot_time_count(tmp, 'Death rate after 500th confirmed case', ['date', 'Country_Region'], legend=True)

It looks like different countries have different trends in the death rate, we thus align them to compare more easily.

In [ ]:
tmp = start_from(conf_cases, deaths, 'from_500th', n_top=10, on_cases=True, make_rate=True, ch_date=True)

fig, ax = plt.subplots(1,1, figsize=(18, 8), facecolor='#f7f7f7') 
fig.suptitle('Death rate from 500th patient', fontsize=18)

tmp = tmp.groupby(['date', 'Country_Region'],as_index=False).n_cases.sum().set_index('date')
tmp.groupby([c for c in ['date', 'Country_Region'] if c != 'date']).n_cases.plot(ax=ax, legend=True)

ax.set_xlabel('Days after 500th confirmed case', fontsize=14)
ax.set_xlim(1,40)

plt.show()

We indeed see different groups:

* Italy, Iran, Spain, and the UK following the same trend and the death rate is increasing steadily  after the 500th confirmed case. It also appears that France, 12 days after the 500th case is following the same trend. 22 days after the 500th case, Iran is slowing down.
* China steadily between 2% and 4%, until day 12 followed by France, who is now joining the Italian curve
* The USA dropping, most likely because more tests were done to patients that were still alive, thus something similar to what happened to Iran in the first days after the 500th case
* Switzerland, South Korea, and Germany with a steady death rate around 1% or less, with Switzerland slowly increasing since day 5.

If we do the same but focusing on the countries with most cases by 1000 inhabitants and a population of at least 1M to avoid extreme values we get

In [ ]:
tmp = pd.merge(conf_cases[['Country_Region', 'date', 'continent', 'n_cases', 'from_500th', 'from_first', 'from_10th', 'population']], 
               deaths[['Country_Region', 'date', 'continent', 'n_cases']], 
               on=['Country_Region', 'date', 'continent'])

tmp['n_cases'] = (tmp['n_cases_y'] / tmp['n_cases_x'] * 100).fillna(0)
tmp_2 = tmp.copy()
tmp_2['n_cases_x'] = tmp_2['n_cases_x'] / tmp_2['population'] * 100
top_countries = tmp_2[tmp_2.population > 1000000].groupby('Country_Region').n_cases_x.max().sort_values(ascending=True).tail(10).index.tolist()
tmp = tmp[(tmp.from_500th > 0)]
tmp = tmp[tmp.Country_Region.isin(top_countries)]
tmp['date'] = tmp['from_500th']

fig, ax = plt.subplots(1,1, figsize=(18, 8), facecolor='#f7f7f7') 
fig.suptitle('Death rate from 500th patient', fontsize=18)

tmp = tmp.groupby(['date', 'Country_Region'],as_index=False).n_cases.sum().set_index('date')
tmp.groupby([c for c in ['date', 'Country_Region'] if c != 'date']).n_cases.plot(ax=ax, legend=True)

ax.set_xlabel('Days after 500th confirmed case', fontsize=14)

plt.show()

We can have a better view of the trends described above and we can also see that Austria is following the numbers of Switzerland, Germany, and Norway. France and Belgium are mirroring one another and Italy, Spain, and the Netherlands are showing the same dramatic trend.

Once again, we want to see how much time is passing between the confirmation of the case and the decease of the patient and again we can only compare the number days between the confirmation of the 1st, 10th, 50th, 100th patient and the 1st, 10th, 50th, 100th death.

In [ ]:
tmp = pd.merge(conf_cases[['Country_Region', 'date', 'continent', 'n_cases', 'first_date', '10th_date', '50th_date', '100th_date']], 
               deaths[['Country_Region', 'date', 'continent', 'n_cases', 'first_date', '10th_date', '50th_date', '100th_date']], 
               on=['Country_Region', 'date', 'continent'])

top_countries = tmp.groupby('Country_Region').n_cases_x.max().sort_values(ascending=True).tail(12).index.tolist()

tmp = tmp[(tmp.Country_Region.isin(top_countries))]

tmp['1st deceased/confirmed'] = (tmp['first_date_y'] - tmp['first_date_x']).dt.days
tmp['10th deceased/confirmed'] = (tmp['10th_date_y'] - tmp['10th_date_x']).dt.days
tmp['50th deceased/confirmed'] = (tmp['50th_date_y'] - tmp['50th_date_x']).dt.days
tmp['100th deceased/confirmed'] = (tmp['100th_date_y'] - tmp['100th_date_x']).dt.days

fig, ax = plt.subplots(1,1, figsize=(10, 18), facecolor='#f7f7f7')
fig.suptitle('Days between confirmed case and death', fontsize=18)
fig.subplots_adjust(top=0.95)

tmp.groupby('Country_Region')[['1st deceased/confirmed', '10th deceased/confirmed', 
                               '50th deceased/confirmed', '100th deceased/confirmed']].min().plot(kind='barh', ax=ax)

ax.axvline(tmp.groupby('Country_Region')[['1st deceased/confirmed', 
                                          '10th deceased/confirmed', 
                                          '50th deceased/confirmed', 
                                          '100th deceased/confirmed']].min().mean().mean(), linestyle='--', color='k')

ax.set_xlabel('Days from case to decease', fontsize=14)

plt.show()

We see that on average the recovery takes slightly less than the demise. We again see longer times for Germany and much shorter ones for Italy, Iran, Spain, and China.

# The Italian Case

Italy is currently the country with the most deaths and the biggest outbreak of Europe. Moreover, the daily press release is fairly detailed with data, containing crucial information such as the number of tests performed, the number of hospitalized patients, etc. Therefore, we can try to perform a more detailed analysis.

The first cases not related to tourism come from the small town of Codogno in Lombardia at the end of February, two days later the region declared the first red zone and closed the schools in the rest of the cities. Since then, containment measures increased every week and they are now a nation-wide lockdown.

First, we can have a look at the national trends once again

In [ ]:
df = pd.read_csv('/kaggle/input/covid19-in-italy/covid19_italy_region.csv')

ignore = ['Country', 'RegionCode', 'Latitude', 'Longitude', 'SNo']
df = df[[col for col in df if col not in ignore]].copy()

df['Date'] = pd.to_datetime(df['Date']).dt.date

df.head()

In [ ]:
tmp = df.groupby('Date').sum()

fig, ax = plt.subplots(1,1, figsize=(18, 8), facecolor='#f7f7f7')
fig.suptitle('Total Cases in Italy', fontsize=18)

tmp.CurrentPositiveCases.plot(label='Current Cases', color='k')
tmp.Deaths.plot(label='Deceased patients', color='r')
tmp.Recovered.plot(label='Recovered patients', color='g')

ax.legend()
plt.xticks(rotation=45)

plt.show()

As before, these numbers are heavily influenced by the number of tests performed. We thus focus on the ratio of patients that tested positive and consequently the ratio of patients that died or recovered.

In [ ]:
tmp['Positivity rate'] = tmp.TotalPositiveCases / tmp.TestsPerformed * 100
tmp['Death rate'] = tmp.Deaths / tmp.TotalPositiveCases * 100
tmp['Recovery rate'] = tmp.Recovered / tmp.TotalPositiveCases * 100

fig, ax = plt.subplots(1,1, figsize=(18, 8), facecolor='#f7f7f7')
fig.suptitle('Positivy, Recovery, Death rates in Italy', fontsize=18)

tmp['Positivity rate'].plot(color='k')
tmp['Death rate'].plot(color='r')
tmp['Recovery rate'].plot(color='g')

ax.legend()
plt.xticks(rotation=45)

plt.show()

We observe how the death rate steadily increased in March from 2-3% to the current 10-12%. As I am writing this there are currently a lot of doubts about these numbers as with the hospitals close to full capacity many people are unfortunately dying in their homes without receiving the test. On the other hand, another debate is about the lack of testing and the suspect is that the number of positive cases is 3 to 10 times higher than the official number (bringing down the fatality rate).

On this last point, we have the data to check that argument. To avoid fluctuations due to data transmission, we take the mean of the new tests performed every 3 days.

In [ ]:
tmp['New Tests'] = tmp.TestsPerformed.diff().fillna(0)

fig, ax = plt.subplots(1,1, figsize=(18, 8), facecolor='#f7f7f7')
fig.suptitle('New tests and positive cases per day in Italy', fontsize=18)

ax = plot_resample(tmp, 'New Tests', ax, 'k', 'New Tests')
ax2 = ax.twinx()
ax2 = plot_resample(tmp, 'NewPositiveCases', ax2, 'r', 'New Positive Cases')

custom_lines = [Line2D([0], [0], color='k', lw=4),
                Line2D([0], [0], color='r', lw=4)]
ax.legend(custom_lines, ['New Tests', 'New Positive Cases'])

ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('Number of tests', fontsize=14)
ax2.set_ylabel('Number of Positive Cases', fontsize=14, color='r')

plt.show()

From this dataset, it appears that the number of new tests has been increasing almost every day and, following the same trend with 10-20% of positivity rate, so did the number of new confirmed cases. Nothing surprising, but there is no element to support that Italy started doing fewer tests as the emergency got bigger. Around March 20th, the positivity rate stop increasing and we can indeed see that the number of new case has been decreasing since then.

We can then see how the emergency got bigger over time (we take averages over a period of 3 days to avoid too many fluctuations).

In [ ]:
tmp['NewDeaths'] = tmp.Deaths.diff().fillna(0)
tmp['NewRecovered'] = tmp.Recovered.diff().fillna(0)
tmp['DeathIncreaseRatio'] = tmp.NewDeaths / (tmp.Deaths - tmp.NewDeaths) * 100
tmp['RecoveredIncreaseRatio'] = tmp.NewRecovered / (tmp.Recovered - tmp.NewRecovered) * 100
tmp['CasesIncreaseRatio'] = tmp.NewPositiveCases / (tmp.TotalPositiveCases - tmp.NewPositiveCases) * 100
tmp.loc[tmp.CasesIncreaseRatio > 200, 'CasesIncreaseRatio'] = np.nan
tmp.loc[tmp.RecoveredIncreaseRatio > 200, 'RecoveredIncreaseRatio'] = np.nan

fig, ax = plt.subplots(2,1, figsize=(18, 18), facecolor='#f7f7f7')
fig.subplots_adjust(top=0.95)
fig.suptitle('Daily increase in Italy', fontsize=18)

ax[0] = plot_resample(tmp, 'NewPositiveCases', ax[0], 'k', 'New Positive Cases')
ax[0] = plot_resample(tmp, 'NewDeaths', ax[0], 'r', 'New Victims')
ax[0] = plot_resample(tmp, 'NewRecovered', ax[0], 'g', 'New Recovered')
ax[1] = plot_resample(tmp, 'CasesIncreaseRatio', ax[1], 'k', 'New Positive Cases')
ax[1] = plot_resample(tmp, 'DeathIncreaseRatio', ax[1], 'r', 'New Victims')
ax[1] = plot_resample(tmp, 'RecoveredIncreaseRatio', ax[1], 'g', 'New Recovered')

ax[0].legend()
ax[1].legend()
ax[0].set_xlabel('')
ax[1].set_xlabel('')
ax[0].set_title('Total New Cases', fontsize=14)
ax[1].set_title('Increase Ratio', fontsize=14)

plt.show()

We see how the increase in new cases dropped from around 30% to the current 5% and how the trend seems to be reversed since March 20th. The same can be said about the percentage of newly deceased patients over the total deceased patients, going from about 20% to 40% in March and then to the current 10%. Two good news. Later we will see if we can attribute this good news to the containment measures or not.

The emergency that Italy is facing is primarily visible in the healthcare system. Insufficient tests for healthcare professionals, lack of personal protection, and overwhelming influx of patients lead, as I am writing this, to 87 doctors that passed due to Coronavirus, many of which are family doctors (thus outside of hospitals). Therefore, the next plot is showing the ratio of positive cases that where home confined or hospitalized. Of this last group, we can see also the ratio of patients that were sent to intensive care.

In [ ]:
tmp['HospitalizedRatio'] = tmp['TotalHospitalizedPatients'] / tmp['CurrentPositiveCases'] * 100
tmp['HomeConfinementRatio'] = tmp['HomeConfinement'] / tmp['CurrentPositiveCases'] * 100
tmp['ICRatio'] = tmp['IntensiveCarePatients'] / tmp['TotalHospitalizedPatients'] * 100

fig, ax = plt.subplots(1,1, figsize=(18, 8), facecolor='#f7f7f7')
fig.suptitle('Hospitalization Ratios', fontsize=18)

tmp['HospitalizedRatio'].plot(color='k')
tmp['ICRatio'].plot(color='r')
tmp['HomeConfinementRatio'].plot(color='g')

ax.legend()
plt.xticks(rotation=45)

plt.show()

We see that in the period between the first week of March and March 20th, a large fraction of positive cases were hospitalized. This might be due to a propensity to hospitalize everyone that needed help or that most of the tests were performed to hospitalized patients. On the other hand, the ratio of hospitalized patients in the ICU is going down from 20% to the current 13%.

## Regions

The initial outbreak started in Lombardia, which luckily has a very good healthcare system but is also heavily densely populated (10 million inhabitants, 422 per squared Km). It can thus easily seen how much of a difference this makes in the numbers

In [ ]:
tmp = df.groupby(['Date', 'RegionName'], as_index=False).sum().set_index('Date')

fig, ax = plt.subplots(3,1, figsize=(18, 24), facecolor='#f7f7f7', sharex=True)
fig.subplots_adjust(top=0.95)
fig.suptitle('Total Cases in Italy by Region', fontsize=18)

tmp.groupby('RegionName').CurrentPositiveCases.plot(legend=True, ax=ax[0], title='Positive Cases')
tmp.groupby('RegionName').Deaths.plot(legend=True, ax=ax[1], title='Deceased Patients')
tmp.groupby('RegionName').Recovered.plot(legend=True, ax=ax[2], title='Recovered Patients')

plt.xticks(rotation=45)

plt.show()

We can see that finally, after a month of growth, the number of Current Positive Cases seems to be stable in Lombardia.

If we group them by macro regions, we see the following

In [ ]:
macro_regions = {'Abruzzo': 'South', 
                 'Basilicata': 'South', 
                 'Calabria': 'South', 
                 'Campania': 'South', 
                 'Emilia-Romagna': 'North-East', 
                 'Friuli Venezia Giulia': 'North-East', 
                 'Lazio': 'Center', 
                 'Liguria': 'North-West', 
                 'Lombardia': 'North-West', 
                 'Marche': 'Center', 
                 'Molise': 'South', 
                 'Piemonte': 'North-West', 
                 'P.A. Bolzano': 'North-East', 
                 'P.A. Trento': 'North-East', 
                 'Puglia': 'South', 
                 'Sardegna': 'Islands', 
                 'Sicilia': 'Islands', 
                 'Toscana': 'Center', 
                 'Trentino Alto Adige / Südtirol': 'North-East', 
                 'Umbria': 'Center', 
                 "Valle d'Aosta": 'North-West', 
                 'Veneto': 'North-East'}

df['macro_region'] = df.RegionName.map(macro_regions)
tmp = df.groupby(['Date', 'macro_region'], as_index=False).sum().set_index('Date')

fig, ax = plt.subplots(3,1, figsize=(18, 24), facecolor='#f7f7f7', sharex=True)
fig.subplots_adjust(top=0.95)
fig.suptitle('Total Cases in Italy by Macro-Region', fontsize=18)

tmp.groupby('macro_region').CurrentPositiveCases.plot(legend=True, ax=ax[0], title='Positive Cases')
tmp.groupby('macro_region').Deaths.plot(legend=True, ax=ax[1], title='Deceased Patients')
tmp.groupby('macro_region').Recovered.plot(legend=True, ax=ax[2], title='Recovered Patients')

plt.xticks(rotation=45)

plt.show()

And the current situation can be captured by the following pie chart

In [ ]:
props = tmp[tmp.index == tmp.index.max()][['macro_region', 'CurrentPositiveCases', 'Deaths', 'TestsPerformed']].copy()
props = props.pivot_table(columns='macro_region')#.reset_index()
props['total'] = props.sum(axis=1)
macro_reg = [c for c in props if 'total' not in c]
for col in macro_reg:
    props[col] = props[col] / props['total']
    
props = props.reset_index()
fig, ax = plt.subplots(1,3, figsize=(15, 5), facecolor='#f7f7f7')
#fig.subplots_adjust(top=0.96)
fig.suptitle(f'Proportion of cases in the macro regions on {tmp.index.max()}', fontsize=18)
i=0
for axes in ax:
    axes.pie(props[macro_reg].values[i], labels=macro_reg, autopct='%.0f%%')
    i += 1
    
ax[0].set_title('Current Positive Cases', fontsize=14)
ax[1].set_title('Deceased Patients', fontsize=14)
ax[2].set_title('Test Performed', fontsize=14)
    
plt.show()

We see how the pandemic has hitten mostly the Northern regions: Lombardia, Piemonte, Veneto, and Emilia Romagna. The most hitten region, Lombardia, is geographically at the center of these groups of regions and also at the center of the spread. The total population of these 4 regions is about 24 million.

We can thus see the positivity, death, and recovery rates for these 4 regions.

In [ ]:
tmp = df[df.RegionName.isin(['Lombardia', 'Emilia-Romagna', 'Veneto', 'Piemonte'])].set_index('Date')
tmp['Positivity rate'] = tmp.TotalPositiveCases / tmp.TestsPerformed * 100
tmp['Death rate'] = tmp.Deaths / tmp.TotalPositiveCases * 100
tmp['Recovery rate'] = tmp.Recovered / tmp.TotalPositiveCases * 100

fig, ax = plt.subplots(3,1, figsize=(18, 26), facecolor='#f7f7f7', sharex=True)
fig.subplots_adjust(top=0.95)
fig.suptitle('Positivy, Recovery, Death rates in Lombardia, Piemonte, Veneto, and Emilia Romagna', fontsize=18)

tmp.groupby('RegionName')['Positivity rate'].plot(legend=True, ax=ax[0], title='Positivity Rate')
tmp.groupby('RegionName')['Death rate'].plot(legend=True, ax=ax[1], title='Death Rate')
tmp.groupby('RegionName')['Recovery rate'].plot(legend=True, ax=ax[2], title='Recovery Rate')

plt.xticks(rotation=45)

plt.show()

We can see how Veneto had since the beginning a significantly lower positivity rate (fraction of positive tests). It then makes sense to check how many tests were executed in these regions

In [ ]:
pops = pd.DataFrame({'RegionName': ['Lombardia', 'Emilia-Romagna', 'Veneto', 'Piemonte'], 
                     'population': [10040, 4453, 4905, 4376]})
tmp = df.copy()
tmp = pd.merge(tmp, pops, on='RegionName')
tmp = tmp.set_index('Date')
tmp['Tests_per1000'] = tmp['TestsPerformed'] / tmp['population']

fig, ax = plt.subplots(2,1, figsize=(18, 16), facecolor='#f7f7f7', sharex=True)
fig.subplots_adjust(top=0.93)
fig.suptitle('Tests performed in Lombardia, Piemonte, Veneto, and Emilia Romagna', fontsize=18)

tmp.groupby('RegionName')['TestsPerformed'].plot(legend=True, ax=ax[0], title='Total Tests')
tmp.groupby('RegionName')['Tests_per1000'].plot(legend=True, ax=ax[1], title='Tests per 1000 inhabitants')

plt.xticks(rotation=45)

plt.show()

In absolute numbers, Lombardia has been testing more patients but it evident the different strategies that Veneto has been following since the beginning. I can't find reliable sources about how accurate the test is, some newspaper reports a high false-negative ratio, others a high false-positive ratio. However, we can see that the death rate in Veneto is lower than in the other regions and we can't exclude this is a consequence of the more intense testing policy.

In [ ]:
test = df[df.RegionName.isin(['Lombardia', 'Emilia-Romagna', 'Veneto', 'Piemonte'])].sort_values(by=['RegionName', 'Date']).set_index('Date')
test['NewTests'] = test.TestsPerformed.diff().fillna(0)
test.loc[test.NewTests < 0, 'NewTests'] = 0

fig, ax = plt.subplots(4,1, figsize=(18, 33), facecolor='#f7f7f7')
fig.subplots_adjust(top=0.95)
fig.suptitle('New tests and positive cases per day in Lombardia, Emilia Romagna, Veneto, and Piemonte', fontsize=18)

i = 0
for region in ['Lombardia', 'Emilia-Romagna', 'Veneto', 'Piemonte']:
    ax[i] = plot_resample(test[test.RegionName==region], 'NewTests', ax[i], 'k', 'New Tests')
    ax2 = ax[i].twinx()
    ax2 = plot_resample(test[test.RegionName==region], 'NewPositiveCases', ax2, 'r', 'New Positive Cases')
    ax[i].set_xlabel('Date', fontsize=14)
    ax[i].set_ylabel('Number of tests', fontsize=14)
    ax2.set_ylabel('Number of Positive Cases', fontsize=14, color='r')
    custom_lines = [Line2D([0], [0], color='k', lw=4),
                Line2D([0], [0], color='r', lw=4)]
    ax[i].legend(custom_lines, ['New Tests', 'New Positive Cases'])
    ax[i].set_title(region, fontsize=14)
    i += 1

plt.show()

We can indeed see how in Lombardia and Piemonte the plots are basically overlapping, maybe denoting a good targeting on who was receiving the test or that the clear inconsistency in the delivery of the tests (or of the data) is biasing the numbers. When the alleged peak was reached, the number of tests stabilized while the number of new cases dropped. On the other hand, in Veneto the number of positive cases, for 2 weeks, was growing quicker than the number of tests performed. We also see how they started testing less in Emilia Romagna, which could be the reason why the number of cases is dropping.

In [ ]:
tmp = df[df.RegionName.isin(['Lombardia', 'Emilia-Romagna', 'Veneto', 'Piemonte'])].sort_values(by=['RegionName', 'Date']).set_index('Date')
tmp['NewDeaths'] = tmp.Deaths.diff().fillna(0)
tmp.loc[tmp.NewDeaths < 0, 'NewDeaths'] = 0
tmp['NewRecovered'] = tmp.Recovered.diff().fillna(0)
tmp.loc[tmp.NewRecovered < 0, 'NewRecovered'] = 0
tmp['DeathIncreaseRatio'] = tmp.NewDeaths / (tmp.Deaths - tmp.NewDeaths) * 100
tmp['RecoveredIncreaseRatio'] = tmp.NewRecovered / (tmp.Recovered - tmp.NewRecovered) * 100
tmp['CasesIncreaseRatio'] = tmp.NewPositiveCases / (tmp.TotalPositiveCases - tmp.NewPositiveCases) * 100
tmp.loc[tmp.CasesIncreaseRatio > 200, 'CasesIncreaseRatio'] = np.nan
tmp.loc[tmp.RecoveredIncreaseRatio > 200, 'RecoveredIncreaseRatio'] = np.nan

fig, ax = plt.subplots(4,2, figsize=(18, 20), facecolor='#f7f7f7', sharex=True)
fig.subplots_adjust(top=0.93)
fig.suptitle('Daily increase in Lombardia, Emilia Romagna, Veneto, and Piemonte', fontsize=18)
i= 0

for region in ['Lombardia', 'Emilia-Romagna', 'Veneto', 'Piemonte']:
    ax[i][0] = plot_resample(tmp[tmp.RegionName == region], 'NewPositiveCases', ax[i][0], 'k', 'New Positive Cases', resample='5D')
    ax[i][1] = plot_resample(tmp[tmp.RegionName == region], 'CasesIncreaseRatio', ax[i][1], 'k', 'New Positive Cases', resample='5D')
    ax[i][0] = plot_resample(tmp[tmp.RegionName == region], 'NewDeaths', ax[i][0], 'r', 'New Victims', resample='5D')
    ax[i][1] = plot_resample(tmp[tmp.RegionName == region], 'DeathIncreaseRatio', ax[i][1], 'r', 'New Victims', resample='5D')
    ax[i][0] = plot_resample(tmp[tmp.RegionName == region], 'NewRecovered', ax[i][0], 'g', 'New Recovered', resample='5D')
    ax[i][1] = plot_resample(tmp[tmp.RegionName == region], 'RecoveredIncreaseRatio', ax[i][1], 'g', 'New Recovered', resample='5D')

    ax[i][0].legend()
    ax[i][1].legend()
    ax[i][0].set_xlabel('')
    ax[i][1].set_xlabel('')
    ax[i][0].set_title(f'{region} - Total New Cases', fontsize=14)
    ax[i][1].set_title(f'{region} - Increase Ratio', fontsize=14)
    
    i += 1

plt.show()

We indeed see how in all the regions the pandemic is slowing down since March 20th. With the exception of Piemonte, it looks like the growth has started again.

In Italy, the healthcare system is managed at the Regional level. We could focus on how the public healthcare system got defunded systematically in the past 20 years (apparently in the same way it was defunded in the rest of Europe) but for now I can't find reliable data. What we can do is to check if the hospitalization policy is consistent across the regions.

In [ ]:
tmp['HospitalizedRatio'] = tmp['TotalHospitalizedPatients'] / tmp['CurrentPositiveCases'] * 100
tmp['HomeConfinementRatio'] = tmp['HomeConfinement'] / tmp['CurrentPositiveCases'] * 100
tmp['ICRatio'] = tmp['IntensiveCarePatients'] / tmp['TotalHospitalizedPatients'] * 100

fig, ax = plt.subplots(2,2, figsize=(18, 10), facecolor='#f7f7f7', sharex=True)
fig.suptitle('Hospitalization Ratios', fontsize=18)

i = 0
j = 0
for region in ['Lombardia', 'Emilia-Romagna', 'Veneto', 'Piemonte']:
    tmp[tmp.RegionName == region]['HospitalizedRatio'].plot(color='k', ax=ax[i][j])
    tmp[tmp.RegionName == region]['ICRatio'].plot(color='r', ax=ax[i][j])
    tmp[tmp.RegionName == region]['HomeConfinementRatio'].plot(color='g', ax=ax[i][j])
    ax[i][j].legend()
    ax[i][j].set_title(region, fontsize=14)
    if j == 1:
        i += 1
        j = 0
    elif j == 0:
        j += 1
    
plt.xticks(rotation=45)

plt.show()

All 4 regions are approaching the emergency differently, 
* Lombardia is hospitalizing more often than not and around 15% of the patients are in the ICU
* Emilia Romagna is recently focusing more on home isolation and only 10% of the hospitalized are in the ICU
* Veneto is overwhelmingly not hospitalizing but 20% of these are in the ICU
* Piemonte is following the pattern of Lombardia

This might depend on beds availability, the criticalness of patients (more patients means more critical one too), or a genuine different policy for hospitalization.

## Containment measures

What follows is the story of containmente measures adopted by Italy from Jan 31st to March 20th (last available date in the data provided)

* Jan 31st: First cases from 2 tourists, all flights from and to China are suspended, thermal scanners in the airports for international passengers
* Feb 6th: an Italian coming back from Wuhan tested positive to COVID-19
* **Feb 22nd**: all public events are canceled, commercial activities (restaurants, bars, etc) have to close at 6 pm. People are advised to not go to the ER if sick but call the doctor first. 10 municipalities in Lombardia, Veneto and Emilia Romagna close the schools. 50k people in lockdown in those regions, local transport is now not stopping in those cities
* Feb 23rd: Social distancing in Lombardia, educational trips are canceled, universities in the North of Italy are closed
* Feb 24th: Basilicata requires a 14 days quarantine for those coming from the North of Italy. Some residents of the red zone violate the quarantine. Court proceedings are postponed. More regions start closing the schools. 31 more laboratories can test for COVID-19. Major companies allow remote working.
* Feb 25th: Multiple fairs and exhibitions were rescheduled. The US bans flights coming from Lombardia. Checkpoints of the red zone are more than doubled
* Feb 26th: multiple regions in the South and Center of Italy close the schools.
* Feb 27th: Italy stops reporting on asymptomatic cases.
* **March 1st**: Red zone in 11 municipalities (quarantine), Yellow zone in Lombardia, Veneto and Emilia Romagna (no social and sports events, schools, theaters, clubs, cinema are closed). In the rest of the country prevention measures are advertised
* March 2nd: remote schooling is now allowed by the Ministry of Education
* March 4th: 100 deaths in the country, all schools are closed, all sports events have to play behind closed doors
* **March 8th**: 16MM people in lockdown in Lombardia, Veneto, Emilia Romagna, Piemonte, e Marche. Those areas can be left only for emergencies or proven working needs. All public events (including religious events) are canceled. People are asked to avoid leaving their homes. At this stage, riots started to break out in a few prisons due to the cancellation of conjugal visits, several inmates died in the riots. In the night, several people left the red zone before it was implemented
* **March 9th**: The red zone is extended to the entire country. Sports events are canceled with exception of clubs participating in international competitions. It is suspected that the outbreak in Bergamo and Valencia are to be attributed to the Atalanta-Valencia game in the Champions League of that day.
* March 11th: all commercial and retail businesses except those providing essential services are closed.
* March 16th: some regions stricken the quarantine for some municipalities 
* March 19th: the army is deployed in Bergamo to help with the emergency
* **March 20th**: free movement is further restricted. All sport can be done only individually and in close proximity of one's residence. Parks and playgrounds are closed down. Any movement towards a residence different from the main one is banned

There were more measures that took place in the last week, in particular further closure of non essential industries but I will wait for the dataset to be updated.

In [ ]:
italy_cont = country_cont(containment, 'Italy')
italy_cont.head()

We will try to find the effects of the containment measures implemented on March 1st (Yellow zone), March 9th (Red zone), and March 20th (further restrictions).

The difficulty in analyzing the containment measures is that there is a delay between the implementation of a measure and its effects. If we look at the number of positive cases, this delay will depend on how much it takes to show symptoms, how quickly you get tested, how quickly you get the results, how quickly it gets reported. Until we have better information, we can assume a delay of **7 days** between a containment measure and its effects on the number of cases, and **further 7 days** (thus 14) to see the effect on the number of deceases.

In [ ]:
tmp = df.groupby('Date').sum()
tmp['NewCasesPerc'] = tmp['NewPositiveCases'] / (tmp.TotalPositiveCases - tmp.NewPositiveCases) * 100
tmp.loc[tmp.index < pd.to_datetime('2020-02-25'), 'NewCasesPerc'] = 0

avg_beforeyellow = tmp[(tmp.index < pd.to_datetime('2020-03-1') - timedelta(days=7)) & (tmp.index < pd.to_datetime('2020-03-1'))].NewCasesPerc.mean()
avg_beforeyellow = tmp[(tmp.index < pd.to_datetime('2020-03-1') - timedelta(days=7)) & (tmp.index < pd.to_datetime('2020-03-1'))].NewCasesPerc.mean()

fig, ax = plt.subplots(3,1, figsize=(15, 15), facecolor='#f7f7f7', sharex=True)
fig.subplots_adjust(top=0.93)
fig.suptitle('Total Cases in Italy', fontsize=18)

ax[0] = plot_resample(tmp, 'TotalPositiveCases', ax[0], 'k', '', index=True, resample='3D')
ax[1] = plot_resample(tmp, 'NewPositiveCases', ax[1], 'k', '', index=True, resample='3D')
ax[2] = plot_resample(tmp, 'NewCasesPerc', ax[2], 'k', '', index=True, resample='3D')
ax[0].set_title('Total Cases', fontsize=13)
ax[1].set_title('New Cases', fontsize=13)
ax[2].set_title('New Cases %', fontsize=13)

max_date = tmp.index.max()

for axes in ax:
    axes = make_areas(axes, max_date)
    
ax = get_avg_beforemeasure(tmp, 'NewCasesPerc', ax)

plt.show()

In the third plot, the one with the relative variation, the horizontal lines are showing the average value in the 7 days prior to the implementation of the containment measure. The decline in the increase seems to speed up.
* When the yellow zone was implemented, on average there were 28% of new cases a day
* When the red zone was implemented, shortly after when we expect the effect of the yellow zone to kick in, there were about 20% of new cases a day
* When the movement was further restricted, well after the effects of both the yellow and red zone kicked in, there were about 12% new cases a day
* As I am writing, the last day has seen 5% of new cases

We can see how the implementation of the red zone seems to have stabilized the number and, hopefully, will start making it go down soon.

To see an effect on the number of deceased patients we assume we need to wait 14 days.

In [ ]:
tmp['NewDeaths'] = tmp.Deaths.diff().fillna(0)
tmp['DeathIncreaseRatio'] = tmp.NewDeaths / (tmp.Deaths - tmp.NewDeaths) * 100

fig, ax = plt.subplots(3,1, figsize=(15, 15), facecolor='#f7f7f7', sharex=True)
fig.subplots_adjust(top=0.93)
fig.suptitle('Total Deceased in Italy', fontsize=18)

ax[0] = plot_resample(tmp, 'Deaths', ax[0], 'k', '', index=True, resample='3D')
ax[1] = plot_resample(tmp, 'NewDeaths', ax[1], 'k', '', index=True, resample='3D')
ax[2] = plot_resample(tmp, 'DeathIncreaseRatio', ax[2], 'k', '', index=True, resample='3D')
ax[0].set_title('Total Deceased', fontsize=13)
ax[1].set_title('New Deceased', fontsize=13)
ax[2].set_title('New Deceased %', fontsize=13)

max_date = tmp.index.max()

for axes in ax:
    axes = make_areas(axes, max_date, offset=14)
    
ax = get_avg_beforemeasure(tmp, 'DeathIncreaseRatio', ax)

plt.show()

We see that shortly after the moment we expect to see the effect of the yellow zone start dropping consistently. Once that the red zone is also implemented, this effect became even more visible.

The primary goal of containment measures like these is to reduce the number of hospitalized patients. We again assume 7 days for the effects to be visible

In [ ]:
tmp['NewHospitalized'] = tmp.TotalHospitalizedPatients.diff().fillna(0)
tmp['NewHospPerc'] = tmp.NewHospitalized / (tmp.TotalHospitalizedPatients - tmp.NewHospitalized) * 100

fig, ax = plt.subplots(3,1, figsize=(15, 15), facecolor='#f7f7f7', sharex=True)
fig.subplots_adjust(top=0.93)
fig.suptitle('Total Hospitalized in Italy', fontsize=18)

ax[0] = plot_resample(tmp, 'TotalHospitalizedPatients', ax[0], 'k', '', index=True, resample='3D')
ax[1] = plot_resample(tmp, 'NewHospitalized', ax[1], 'k', '', index=True, resample='3D')
ax[2] = plot_resample(tmp, 'NewHospPerc', ax[2], 'k', '', index=True, resample='3D')
ax[0].set_title('Total Hospitalized', fontsize=13)
ax[1].set_title('New Hospitalized', fontsize=13)
ax[2].set_title('New Hospitalized %', fontsize=13)

max_date = tmp.index.max()

for axes in ax:
    axes = make_areas(axes, max_date, offset=7)
    
ax = get_avg_beforemeasure(tmp, 'NewHospPerc', ax)

plt.show()

We indeed see that the implementation of the red zone seems to have been particularly effective in reducing the influx of patients into the hospitals. Unfortunately other reasons can justify this trend inversion:
* different hospitalization policies
* the hospitals are full and can't accept new patients

However, the trend stayed the same for several days and the most obvious explanation is that the containment measure is working.

If we focus on the IC patients we also see the growth slowing down but, on top of the two possible explanations, given above we have to consider that as *New IC Patients* we merely describe the difference between one day and the previous one, thus new patients minus patients that freed the bed (either because they recovered or because they didn't make it).

In [ ]:
tmp['NewIC'] = tmp.IntensiveCarePatients.diff().fillna(0)
tmp['NewICPerc'] = tmp.NewIC / (tmp.IntensiveCarePatients - tmp.NewIC) * 100

fig, ax = plt.subplots(3,1, figsize=(15, 15), facecolor='#f7f7f7', sharex=True)
fig.subplots_adjust(top=0.93)
fig.suptitle('IC Patients in Italy', fontsize=18)

ax[0] = plot_resample(tmp, 'IntensiveCarePatients', ax[0], 'k', '', index=True, resample='3D')
ax[1] = plot_resample(tmp, 'NewIC', ax[1], 'k', '', index=True, resample='3D')
ax[2] = plot_resample(tmp, 'NewICPerc', ax[2], 'k', '', index=True, resample='3D')
ax[0].set_title('Current IC Patients', fontsize=13)
ax[1].set_title('New IC Patients', fontsize=13)
ax[2].set_title('New IC Patients %', fontsize=13)

max_date = tmp.index.max()

for axes in ax:
    axes = make_areas(axes, max_date, offset=7)
    
ax = get_avg_beforemeasure(tmp, 'NewICPerc', ax)

plt.show()

Given that the growth is still going on, can temporarily try to fit an exponential curve on different sections of this data to simulate what would have happened if no containment measure was implemented.

In [ ]:
fig, ax = plt.subplots(3,1, figsize=(15, 15), facecolor='#f7f7f7', sharex=True)
fig.subplots_adjust(top=0.92)
fig.suptitle('Total Expected case without containment', fontsize=18)

ax[0] = plot_fits(tmp, 'TotalPositiveCases', ax[0], offset=10)
ax[1] = plot_fits(tmp, 'Deaths', ax[1], offset=14)
ax[2] = plot_fits(tmp, 'TotalHospitalizedPatients', ax[2], offset=10)
ax[0].set_title('Confirmed Cases', fontsize=14)
ax[1].set_title('Deceased Patients', fontsize=14)
ax[2].set_title('Hospitalized Patients', fontsize=14)
ax[0].legend()
ax[1].legend()
ax[2].legend()

plt.show()

According to this simulation, which has a very limiting assumption on the shape of the curve (a sigmoid would be more appropriate but I am encountering some problems in fitting now), the implementation of the Yellow Zone on March 1st has reduced the number of cases to 1/5, the number of deceased patients to 1/6, and the number of hospitalized to less than 1/10.

# Similar Countries

After studying the Italian situation, a country that is ahead of the curve with respect to other countries, it is worth looking for similarities with other countries. In the provided data, we have 2 pieces of information regarding population density and median age. It must be said that Countries like China or the USA have a very diverse territory that changes a lot the population density (cities vs countriside), so we should not focus too much on those numbers.

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15, 6), facecolor='#f7f7f7')

tmp = conf_cases[conf_cases.pop_density < 1000]
tmp.pop_density.hist(bins=50, ax=ax[0], color='peru', alpha=0.7)
tmp.median_age.hist(bins=10, ax=ax[1], color='seagreen', alpha=0.7)
ax[0].grid(False)
ax[1].grid(False)
ax[0].annotate('Italy',
            xy=(tmp[tmp.Country_Region=='Italy'].pop_density.max(), 200), 
            xycoords='data', xytext=(-25, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax[0].annotate('Netherlands',
            xy=(tmp[tmp.Country_Region=='Netherlands'].pop_density.max(), 100), 
            xycoords='data', xytext=(-25, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax[0].annotate('UK',
            xy=(tmp[tmp.Country_Region=='United Kingdom'].pop_density.max(), 100), 
            xycoords='data', xytext=(25, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax[0].annotate('China',
            xy=(tmp[tmp.Country_Region=='China'].pop_density.max(), 100), 
            xycoords='data', xytext=(-35, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax[0].annotate('Spain',
            xy=(tmp[tmp.Country_Region=='Spain'].pop_density.max(), 600), 
            xycoords='data', xytext=(35, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax[0].annotate('South Africa',
            xy=(tmp[tmp.Country_Region=='South Africa'].pop_density.max(), 820), 
            xycoords='data', xytext=(35, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax[0].annotate('Germany',
            xy=(tmp[tmp.Country_Region=='Germany'].pop_density.max(), 220), 
            xycoords='data', xytext=(35, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))

ax[1].annotate('Italy',
            xy=(tmp[tmp.Country_Region=='Italy'].median_age.max(), 620), 
            xycoords='data', xytext=(-25, -35), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax[1].annotate('Netherlands',
            xy=(tmp[tmp.Country_Region=='Netherlands'].median_age.max(), 1100), 
            xycoords='data', xytext=(-25, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax[1].annotate('UK',
            xy=(tmp[tmp.Country_Region=='United Kingdom'].median_age.max(), 900), 
            xycoords='data', xytext=(-25, -25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax[1].annotate('China',
            xy=(tmp[tmp.Country_Region=='China'].median_age.max(), 1000), 
            xycoords='data', xytext=(-25, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax[1].annotate('Spain',
            xy=(tmp[tmp.Country_Region=='Spain'].median_age.max(), 630), 
            xycoords='data', xytext=(15, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax[1].annotate('South Africa',
            xy=(tmp[tmp.Country_Region=='South Africa'].median_age.max(), 1200), 
            xycoords='data', xytext=(15, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax[1].annotate('Germany',
            xy=(tmp[tmp.Country_Region=='Germany'].median_age.max(), 200), 
            xycoords='data', xytext=(-15, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))


ax[0].set_title('Population Density', fontsize=16)
ax[1].set_title('Median Age', fontsize=16)
ax[0].set_xlabel('Pop/Km2', fontsize=14)
ax[1].set_xlabel('Age', fontsize=14)

plt.show()

Italy has a high population density and one of the highest median age in the world. We can thus look for patterns in relation with the number of positive cases and deceased patients. Limiting the analysis to the countries with at least 1000 cases or at least 10 deceased patients, we get

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(15, 15), facecolor='#f7f7f7')

cases = conf_cases[conf_cases.pop_density < 1000]
dea = deaths[deaths.pop_density < 1000]
tmp = pd.merge(cases[['Country_Region', 'date', 'n_cases', 'population', 'pop_density', 'median_age']], 
               dea[['Country_Region', 'date', 'n_cases', 'population', 'pop_density', 'median_age']], 
               on=['Country_Region', 'date', 'population', 'pop_density', 'median_age'])
tmp['n_cases'] = tmp['n_cases_y'] / tmp['n_cases_x'] * 100
cases = cases[cases.date == cases.date.max()][['Country_Region', 'n_cases', 'population', 'pop_density', 'median_age']].drop_duplicates()
cases = cases[cases.n_cases > 1000]
# dea = dea[dea.date == dea.date.max()][['Country_Region', 'n_cases', 'population', 'pop_density', 'median_age']].drop_duplicates()
# dea = dea[dea.n_cases > 10]
tmp = tmp[tmp.n_cases_y > 10]
tmp = tmp[tmp.date == tmp.date.max()][['Country_Region', 'n_cases', 'population', 'pop_density', 'median_age']].drop_duplicates()


ax[0][0].scatter(cases.pop_density, cases.n_cases/cases.population*1000, s=cases.population/1000000, color='lightsteelblue', alpha=0.9)
ax[0][1].scatter(tmp.pop_density, tmp.n_cases, s=tmp.population/1000000, color='lightsteelblue', alpha=0.9)
ax[1][0].scatter(cases.median_age, cases.n_cases/cases.population*1000, s=cases.population/1000000, color='lightsteelblue', alpha=0.9)
ax[1][1].scatter(tmp.median_age, tmp.n_cases, s=tmp.population/1000000, color='lightsteelblue', alpha=0.9)

countries = {'Italy': 'r', 'Netherlands': 'darkorange', 'Spain': 'y', 'United Kingdom': 'dodgerblue', 'South Africa': 'g', 'India': 'pink', 'Germany': 'k'}

for country in countries:
    ax[0][0] = plot_country(cases, country, 'pop_density', ax[0][0], countries[country], cases=True)
    ax[0][1] = plot_country(tmp, country, 'pop_density', ax[0][1], countries[country])
    ax[1][0] = plot_country(cases, country, 'median_age', ax[1][0], countries[country], cases=True)
    ax[1][1] = plot_country(tmp, country, 'median_age', ax[1][1], countries[country])

ax[0][0].set_xlabel('Population Density', fontsize=14)
ax[0][1].set_xlabel('Population Density', fontsize=14)
ax[1][0].set_xlabel('Median Age', fontsize=14)
ax[1][1].set_xlabel('Median Age', fontsize=14)
ax[0][0].set_ylabel('Confirmed Cases x 1000', fontsize=14)
ax[0][1].set_ylabel('Mortality Rate (%)', fontsize=14)
ax[1][0].set_ylabel('Confirmed Cases x 1000', fontsize=14)
ax[1][1].set_ylabel('Mortality Rate (%)', fontsize=14)

plt.show()

Where the size of the points is proportional to the population of the country. 
* Italy in red
* The UK in blue
* Spain in yellow
* South Africa in green
* India in pink 
* The Netherlands in orange
* Germany in black

While there is no evident pattern between confirmed cases or deceased patients with the population density, it might be that the median age is playing a role for the death rate (most of the victims are elderly patients with pre-existing conditions).

In this, Italy and Spain look very close to each other. In reality, if we again focus on the number of confirmed cases per thousand inhabitants after the 10th victim, we notice that Spain is not the only country following the footsteps of Italy

In [ ]:
tmp = start_from(conf_cases, deaths, 'from_10th', on_cases=False, n_top=1000, true_count='x', make_rate=False, ch_date=True)

tmp['n_cases'] = tmp['n_cases'] / tmp['population'] * 1000
tmp = tmp[(tmp.population > 1000000)]

top_countries = ['Italy', 'Netherlands', 'Germany', 'United Kingdom', 'Spain', 'France', 'United States', 'China', 'South Africa']
tmp = tmp[tmp.Country_Region.isin(top_countries)]

fig, ax = plt.subplots(1,1, figsize=(18, 8), facecolor='#f7f7f7') 
fig.suptitle('Confirmed cases per 1000 inhabitants since the 10th deceased patient', fontsize=18)

tmp = tmp.groupby(['date', 'Country_Region'],as_index=False).n_cases.sum().set_index('date')
tmp.groupby([c for c in ['date', 'Country_Region'] if c != 'date']).n_cases.plot(ax=ax, legend=True)

ax.annotate('USA',
            xy=(tmp[tmp.Country_Region=='United States'].reset_index().date.max(), tmp[tmp.Country_Region=='United States'].n_cases.max()), 
            xycoords='data', xytext=(55, 0), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('Italy',
            xy=(tmp[tmp.Country_Region=='Italy'].reset_index().date.max(), tmp[tmp.Country_Region=='Italy'].n_cases.max()), 
            xycoords='data', xytext=(-35, -35), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('Spain',
            xy=(tmp[tmp.Country_Region=='Spain'].reset_index().date.max(), tmp[tmp.Country_Region=='Spain'].n_cases.max()), 
            xycoords='data', xytext=(45, 5), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('Germany',
            xy=(tmp[tmp.Country_Region=='Germany'].reset_index().date.max(), tmp[tmp.Country_Region=='Germany'].n_cases.max()), 
            xycoords='data', xytext=(-25, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('Netherlands',
            xy=(tmp[tmp.Country_Region=='Netherlands'].reset_index().date.max(), tmp[tmp.Country_Region=='Netherlands'].n_cases.max()), 
            xycoords='data', xytext=(15, 15), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('UK',
            xy=(tmp[tmp.Country_Region=='United Kingdom'].reset_index().date.max(), tmp[tmp.Country_Region=='United Kingdom'].n_cases.max()), 
            xycoords='data', xytext=(25, -25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('France',
            xy=(tmp[tmp.Country_Region=='France'].reset_index().date.max(), tmp[tmp.Country_Region=='France'].n_cases.max()), 
            xycoords='data', xytext=(25, 15), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))

ax.set_yscale('log')
ax.set_xlim((1,45))
ax.grid(axis='y')

plt.show()

Putting the graph in the logarithmic scale helps for seeing the variation of the exponential curve that every country is following at the moment, therefore every horizontal line represents 10 times more cases than the previous one. Since comparing the number of positive cases can be misleading if we don't take into account the population size, we consider the cases by thousand of inhabitants. If we start counting from the day of the 10th victim in each country we see how every line is more or less parallel to the one of Italy, meaning that each country is increasing the number of cases similarly to Italy. In particular, France and the Uk are getting closer and closer. Spain was joining them but 5 days after the 10th victim had seen a surge in positive cases. The Netherlands was following the same curve of Spain but then slowed down.

We can also appreciate how all the curves are way steeper than the Chinese one, showing how their containment measures were effective.

Similarly, if we focus on the victims after the 500th confirmed case, again leaving part of the Chinese curve for reference, we get

In [ ]:
tmp = start_from(conf_cases, deaths, 'from_500th', on_cases=True, n_top=1000, true_count='y', make_rate=False, ch_date=True)

top_countries = ['Italy', 'Netherlands', 'Germany', 'United Kingdom', 'Spain', 'France', 'United States', 'China', 'South Africa']
tmp = tmp[tmp.Country_Region.isin(top_countries)]

fig, ax = plt.subplots(1,1, figsize=(18, 8), facecolor='#f7f7f7') 
fig.suptitle('Victims since the 500th confirmed cases', fontsize=18)

tmp = tmp.groupby(['date', 'Country_Region'],as_index=False).n_cases.sum().set_index('date')
tmp.groupby([c for c in ['date', 'Country_Region'] if c != 'date']).n_cases.plot(ax=ax, legend=True)

ax.annotate('USA',
            xy=(tmp[tmp.Country_Region=='United States'].reset_index().date.max(), tmp[tmp.Country_Region=='United States'].n_cases.max()), 
            xycoords='data', xytext=(5, 20), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('Italy',
            xy=(tmp[tmp.Country_Region=='Italy'].reset_index().date.max(), tmp[tmp.Country_Region=='Italy'].n_cases.max()), 
            xycoords='data', xytext=(-35, -35), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('Spain',
            xy=(tmp[tmp.Country_Region=='Spain'].reset_index().date.max(), tmp[tmp.Country_Region=='Spain'].n_cases.max()), 
            xycoords='data', xytext=(55, 5), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('Germany',
            xy=(tmp[tmp.Country_Region=='Germany'].reset_index().date.max(), tmp[tmp.Country_Region=='Germany'].n_cases.max()), 
            xycoords='data', xytext=(25, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('Netherlands',
            xy=(tmp[tmp.Country_Region=='Netherlands'].reset_index().date.max(), tmp[tmp.Country_Region=='Netherlands'].n_cases.max()), 
            xycoords='data', xytext=(45, 15), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('UK',
            xy=(tmp[tmp.Country_Region=='United Kingdom'].reset_index().date.max(), tmp[tmp.Country_Region=='United Kingdom'].n_cases.max()), 
            xycoords='data', xytext=(25, -35), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('France',
            xy=(tmp[tmp.Country_Region=='France'].reset_index().date.max(), tmp[tmp.Country_Region=='France'].n_cases.max()), 
            xycoords='data', xytext=(25, -5), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))
ax.annotate('South Africa',
            xy=(tmp[tmp.Country_Region=='South Africa'].reset_index().date.max(), tmp[tmp.Country_Region=='South Africa'].n_cases.max()), 
            xycoords='data', xytext=(35, 5), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05))

ax.set_yscale('log')
ax.set_xlim((1,43))
ax.grid(axis='y')

plt.show()

About 10 days after the 500th confirmed case, China started seeing a decline in the victims. None of the other countries here displayed is showing such a decline. In particular,
* Italy, Spain, and the UK are following exactly the same line without the decline. Italy, as seen above, is slowing down after the containment measures discussed above. We will see if the other countries will do the same
* France and the USA are following the same curve, which is now faster than the Italian one at that stage, with the USA going a bit quicker.
* The Netherlands is now following the curve of China.
* Germany has a very low death rate but the increase is as steady as the other countries.

There is an evident **difference between Germany and the other countries** here highlighted: a very low death rate even though its median age is comparable with the one of Italy and Spain. 

In [ ]:
tmp = germany.groupby('age_group', as_index=False)[['new_cases', 'new_deaths']].sum()
germ_pop = pd.read_csv('/kaggle/input/covid19-tracking-germany/demographics_de.csv')
germ_pop = germ_pop.groupby('age_group').population.sum()
tmp = pd.merge(tmp, germ_pop, on='age_group')
tmp['positivy_rate'] = tmp['new_cases'] / tmp['population']
tmp['death_rate'] = tmp['new_deaths'] / tmp['new_cases']
tmp['prop_positives'] = tmp['new_cases'] / tmp['new_cases'].sum()
tmp['prop_deaths'] = tmp['new_deaths'] / tmp['new_deaths'].sum()

fig, ax = plt.subplots(3,2, figsize=(15, 15), facecolor='#f7f7f7')
fig.subplots_adjust(top=0.92)
fig.suptitle('Summary of the situation in Germany', fontsize=18)

tmp.set_index('age_group').new_cases.plot(kind='bar', ax=ax[0][0], color='gold')
tmp.set_index('age_group').new_deaths.plot(kind='bar', ax=ax[0][1], color='red')
tmp.set_index('age_group').positivy_rate.plot(kind='bar', ax=ax[1][0], color='gold')
tmp.set_index('age_group').death_rate.plot(kind='bar', ax=ax[1][1], color='red')

ax[2][0].pie(tmp.prop_positives.values, labels=tmp.age_group, autopct='%.0f%%')
ax[2][1].pie(tmp.prop_deaths.values, labels=tmp.age_group, autopct='%.0f%%')

ax[0][0].set_title('Total Cases', fontsize=14)
ax[0][1].set_title('Total Victims', fontsize=14)
ax[1][0].set_title('Positivity Rate (%)', fontsize=14)
ax[1][1].set_title('Death Rate (%)', fontsize=14)
ax[2][0].set_title('Proportion of Positives', fontsize=14)
ax[2][1].set_title('Proportion of Victims', fontsize=14)

for axes in ax[0]:
    axes.set_xlabel('')
    axes.set_xticklabels(axes.get_xticklabels(), rotation=0)
    axes.grid(axis='y')
for axes in ax[1]:
    axes.set_xlabel('')
    axes.set_xticklabels(axes.get_xticklabels(), rotation=0)
    axes.grid(axis='y')
    axes.set_yticklabels(['{:,.2%}'.format(x) for x in axes.get_yticks()])

plt.show()

We indeed see that **the majority of the cases are coming from the 35-59 age group**, which is also, not only in Germany, an age group with a low death rate. It has to be noted that they are the majority of the cases even taking into account that they have also the largest population size, i.e. they also have the highest positivy rate. In other words, it seems that in Germany they **prevented the spread of the disease among the most vulnerable part of the population**: people with more than 60 years. Indeed, more than 90% of the victims in Gemany have more than 60 years of age.

Another interpretation is that Germany is not testing enough the most vulnerable part of the population and thus all those victims are happening under the radar. It doesn't seem likely as the positivity rate is similar in each age group but we can't exclude that possibility using this dataset. As mentioned since the beginning, this is an issue for every country since all the numbers are based on who gets the test. A better count will be possible only at the end by comparing the average number of deaths with the number of deaths of these months.

The world seems to have ignored the lesson that was coming from China. I didn't pick these countries only because they provide enough cases to do some analysis, but also because they are all so-called western countries, most of them share a border and one of them, Italy, is ahead of the curve. The question is now what the other countries did about it.

# Did the rest of the world learn something?

We have seen above that it is reasonable to say that the containment measures adopted in Italy have sorted some visible effect and yet the situation has gotten worse and worse. We have also seen how Italy is a few days or weeks ahead of the other countries. We can thus see if there is some similarity in the containment measures adopted.

We now use the dataset about containment measures, that looks like this

In [ ]:
to_use = ['Country_Region', 'n_cases', 'increase', 'perc_increase', 'date', 'first_date', 'from_first', 
          '10th_date', 'from_10th', '50th_date', 'from_50th', '100th_date', 'from_100th', '500th_date', 'from_500th']

full_data = pd.merge(conf_cases[to_use].rename(columns={'first_date': 'first_case_date', 
                                                        'from_first': 'from_first_case', 
                                                        '10th_date': '10th_case_date', 
                                                        'from_10th': 'from_10th_case', 
                                                        '50th_date': '10th_case_date', 
                                                        'from_50th': 'from_50th_date',
                                                        '100th_date': '100th_case_date', 
                                                        'from_100th': 'from_100th_case', 
                                                        '500th_date': '500th_case_date',
                                                        'from_500th': 'from_500th_case', 
                                                        'increase': 'new_cases', 
                                                        'perc_increase': 'new_cases_perc'}), 
                     deaths[to_use].rename(columns={'first_date': 'first_victim_date', 
                                                        'from_first': 'from_first_victim', 
                                                        '10th_date': '10th_victim_date', 
                                                        'from_10th': 'from_10th_victim', 
                                                        '50th_date': '50th_victim_date', 
                                                        'from_50th': 'from_50th_victim',
                                                        '100th_date': '100th_victim_date', 
                                                        'from_100th': 'from_100th_victim', 
                                                        '500th_date': '500th_victim_date',
                                                        'from_500th': 'from_500th_victim', 
                                                    'n_cases': 'n_victims', 
                                                        'increase': 'new_victims', 
                                                        'perc_increase': 'new_victims_perc'}), 
                    on=['Country_Region', 'date'])


full_data.head()

In [ ]:
cont_all = []

for country in ['Italy', 'Netherlands', 'Germany', 'United Kingdom', 'Spain', 'France', 'United States']:
    tmp = country_cont(containment, country)
    tmp['Country_Region'] = country
    tmp = tmp[tmp.Keywords.str.contains('lockdown|business suspension|school closure|travel ban|social distancing')]
    cont_all.append(tmp[['Country_Region', 'date', 'Keywords', 'region']])
    
cont_all = pd.concat(cont_all, ignore_index=False)
cont_all = cont_all[(cont_all.date >= pd.to_datetime('2020-02-01'))]  # a school ban that I can't find proof of

cont_all.head()

We can thus check when each of the above countries** closed the schools**

In [ ]:
tmp = cont_all[cont_all.Keywords.str.contains('school')].groupby('Country_Region', as_index=False).date.min()

tmp = pd.merge(tmp, full_data, on=['Country_Region', 'date'])
tmp[['Country_Region', 'date', 'n_cases', 'n_victims', 'from_10th_case', 'from_first_victim', 'from_100th_case', 'from_10th_victim']].sort_values('n_cases')

* Italy closed the first schools (in the regions with cases) on February 22nd, a day after the first victim. At the time there were 62 cases and 2 victims, the next day there would be the 100th case, 3 days later the 10th victim
* Spain closed the schools on March 9th, 2 days after the 10th victim and 7 days after the 100th case. There were more than a 1000 cases and 28 victims. In Italy, that day, there were 9172 cases, 463 victims
* The first schools in the USA were closed on March 11th, 7 days after the 10th victim and 8 days after the 100th case. There were 1281 cases and 36 victims. In Italy, there were 12462 cases and 827 victims
* Germany closed the schools on March 13th, 4 days after the first victim and 12 days after the 100th case. There were 3675 cases and 7 victims. In Italy, that day, there were 17660 cases and 1266 victims
* The Netherlands closed the schools on March 15th, 2 days after the 10th victim and 9 days after the 100th case. There were 1138 cases and 20 victims. In Italy, that day, there were 24747 cases and 1809 victims
* France closed the schools on March 16th, 16 days after the 100th case and 9 days after the 10th victim. There were 6683 case and 149 victims. In Italy, that day, there were 27980 cases and 2158 victims
* The UK closed the schools on March 20th, 15 days after the 100th case and 6 days after the 10th victim. There were 4014 cases and 178 victims. In Italy, that day, there were 47021 cases and 4032 victims

If we look at forms of business limitations, like closing restaurant or companies, we see the following

In [ ]:
tmp = cont_all[cont_all.Keywords.str.contains('business')].groupby('Country_Region', as_index=False).date.min()

tmp = pd.merge(tmp, full_data, on=['Country_Region', 'date'])
tmp[['Country_Region', 'date', 'n_cases', 'n_victims', 'from_10th_case', 'from_first_victim', 'from_100th_case', 'from_10th_victim']].sort_values('n_cases')

* Italy did it on the day of the 10th victim, February 25th, 2 days after the 100th case.
* Germany did it on March 14th, 5 days after the first victim and 13 days after the 100th case. There were 4585 cases and 9 victims. Italy was facing 21157 cases and 1441 victims
* Spain did it also on March 14th, 7 days after the 10th victim and 12 days after the 100th case. There were 6391 cases and 195 victims.
* The Netherlands did it on March 15th, 2 days after the 10th victim and 9 days after the 100th case. There were 1138 cases and 20 victims. Italy was facing 24747 cases and 1809 victims
* France did it also on March 15th, 15 days after the 100th case and 8 days after the 10th victim. There were 4532 cases and 91 victims.
* The USA didi it on March 15th, 12 days after the 100th case and 11 days after the 10th victim. There were 3499 cases and 63 victims.

Information about the recent closures of bars and restaurants in the UK is not included in the dataset yet.

In [ ]:
tmp = cont_all[cont_all.Keywords.str.contains('social dista')].groupby('Country_Region', as_index=False).date.min()

tmp = pd.merge(tmp, full_data, on=['Country_Region', 'date'])
tmp[['Country_Region', 'date', 'n_cases', 'n_victims', 'from_10th_case', 'from_first_victim', 'from_100th_case', 'from_10th_victim']].sort_values('n_cases')

The social distancing took place:
* 2 days after the first victim in Italy
* On the same day of the first victim, 2 weeks after Italy
* 10 days after the 10th victim in the USA, 5 days after Germany
* 8 days after the 10th victim in France, the day after the USA
* 2 days after the 10th victim in the UK, a day after France
* 10 days after the 10th victim in the Netherlands, 8 days after the UK

On this last point, technically the advise to stay socially distant came on March 15th in the Netherlands but was ignored by so many people the government had to implement fines and send a message on the emergency channel.

In [ ]:
tmp = cont_all[cont_all.Keywords.str.contains('travel ban')].groupby('Country_Region', as_index=False).date.min()

tmp = pd.merge(tmp, full_data, on=['Country_Region', 'date'])
tmp[['Country_Region', 'date', 'n_cases', 'n_victims', 'from_10th_case', 'from_first_victim', 'from_100th_case', 'from_10th_victim']].sort_values('n_cases')

So far, no other country has implemented a lockdown similar to the Italian one.

In [ ]:
cont_all[cont_all.Keywords.str.contains('lockdown')].groupby('Country_Region', as_index=False).date.min()

***NOTE: some of the containment measure disappeared from the dataset in one of the updates***

We have seen before how the effect of the lockdown is visible in the trend of Italy. It feels like the most effective measure but it also came with the business limitations and the closure of the schools. We can't thus say what was the measure that created that effect. 

We can however say that no other country adopted a similar set of measures (despite calling it a lockdown at times). Moreover, every other country here listed, despite the news that was coming from Italy and the fact that, as we have seen, they are following the same curve, waited as much as they could to implement similar measures like closing restaurants or schools. At that time, the virus was much more spread and we can only hope that those 7/10 days of the advantage they had is not now wasted.

We can summarize the evolution of the pandemic in these countries by looking at the daily new cases and putting them in the context with the first containment measure of each type adopted in each country.

In [ ]:
def get_case(data, col):
    df = data.copy().reset_index()
    date = df[col].max()
    if 'case' in col:
        y_coord = df[df.date == date]['new_cases'].max()
    elif 'victim' in col:
        y_coord = df[df.date == date]['new_victims'].max()
    return date, y_coord


def line_containment(data, measure, ax, color, name=None, legend_lines=None, legend_names=None):
    x_coord = data[data.Keywords.str.contains(measure)].date.min()
    
    try:
        ax.axvline(x_coord, linestyle='--', color=color, alpha=0.5)     
        legend_lines.append(Line2D([0], [0], color=color, lw=2, linestyle='--', alpha=0.5))
        legend_names.append(name)
    except ValueError:
        pass
    
    return ax, legend_lines, legend_names


def country_containment(containment, data, ax):
    tmp = data[data.Country_Region == country].set_index('date')
    tmp_c = containment[containment.Country_Region == country].copy()
    
    tmp.new_cases.plot(color='b', alpha=0.3, ax=ax, label='New Cases')
    ax2 = ax.twinx()
    tmp.new_victims.plot(color='r', alpha=0.3, ax=ax2, label='New Victims')
    
    custom_lines = [Line2D([0], [0], color='b', lw=2, alpha=0.3),
                Line2D([0], [0], color='r', lw=2, alpha=0.3)]
    legend_names = ['New Cases', 'New Victims']
    
    events = ['100th_case_date', '500th_case_date', 'first_victim_date', '10th_victim_date', '100th_victim_date']
    for event in events:
        x_coord, y_coord = get_case(tmp.copy(), event)
        if y_coord is np.nan:
            y_coord = 1
        
        if 'case' in event:
            ax.annotate(event.split('_date')[0].replace('_', ' '),
                xy=(x_coord, y_coord), 
                xycoords='data', xytext=(-55, 22), textcoords='offset points',
                arrowprops=dict(facecolor='black', shrink=0.05))
        else:
            ax2.annotate(event.split('_date')[0].replace('_', ' '),
                xy=(x_coord, y_coord), 
                xycoords='data', xytext=(-55, 42), textcoords='offset points',
                arrowprops=dict(facecolor='black', shrink=0.05))
    
    measures = {'school': ['School cls.', 'k'], 
                'social': ['Soc. Dist.', 'b'], 
                'business': ['Business limit', 'g'],
                'lockdown': ['Lockdown', 'r']}
    for measure in measures:
        ax, custom_lines, legend_names = line_containment(tmp_c, measure, ax, measures[measure][1], measures[measure][0], custom_lines, legend_names)
    
    ax.set_title(country, fontsize=14)
    ax.set_ylabel('New Cases', color='b')
    ax2.set_ylabel('New Victims', color='r')
    
    ax.legend(custom_lines, legend_names)
    
    return ax

In [ ]:
top_countries = ['Italy', 'Netherlands', 'Germany', 'France', 'Spain', 'United Kingdom', 'United States']

full_data['date'] = full_data['date'].dt.date

full_data = full_data[full_data.date > pd.to_datetime('2020-02-15')].copy()

fig, ax = plt.subplots(len(top_countries),1, figsize=(18, 3*len(top_countries)), facecolor='#f7f7f7', sharex=True) 
fig.suptitle('Containmente measures and daily increase', fontsize=18)
fig.subplots_adjust(top=0.95)

i = 0
for country in top_countries:
    ax[i] = country_containment(cont_all, full_data, ax[i])
    i += 1

plt.xticks(rotation=45)
plt.show()

It has to be noted that the first lockdown in Italy was a regional one (and all the cases were in that region) and that the early action did not prevent the spread well enough. Looking at the Italian newspapers, it is due to poor implementation in the early stages. Germany acted soon with the social distancing on the day of the first victim, but we have to remember that the first victim happened relatively late since the beginning of the pandemic. The other countries waited much longer: France and the UK waited for the 100th victim, the Netherlands, Spain, and the USA for the 500th case. A delay that hopefully will not be paid to harshly by the population.

# Conclusions

I think we can all agree that we would like to see this section written down and not changed any longer. Unfortunately, the virus is still spreading and we have seen how quickly the situation can change. Moreover, there is a lot of uncertainty on how the data is being recorded by each country and this can bias the analysis. For this reason, I reserve the originally planned forecast section to another Notebook.

If there is a takeaway message, that would be that every country is at a different point of roughly the same curve, meaning that some countries have been lucky enough to be able to see their own future. Inaction or ineffective action of European countries, starting from Italy to the UK, once they have seen what was happening in China, is unjustifiable and it is costing lives, resources, and future opportunities. As if there is a vague sense of *this is not going to happen to us, we are different*.

Unfortunately, this is not the case, once the virus has spread enough we have seen that in every country, despite the different reporting methods, **the growth has been nearly identical**. It remains open the question of how come for some countries the fatality rate is much higher than in others. We can find some intuitive explanation in looking at the relationship with the median age of a country but this is heavily influenced by how the victims are counted. Even Italy, which is generally regarded as a country reporting every case with a lot of transparency during this crisis, can raise some doubts about underestimating its victims as at some moment the hospitals of certain areas were so full that people were dying at home without receiving a test, at least according to the Italian newspapers and the comparison of the deaths of the last month with respect to the yearly averages.

We need them to wait, monitor the situation unfolding and draw our conclusions a bit further down the line. Hopefully while moving on from petty rivalries and by staying safe all together.